In [7]:
import pyautogui as pag
import gym
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import collections
import random
import matplotlib.pyplot as plt
import pyautogui as pag
import PIL
import torchvision.transforms as transforms
import cv2
import numpy as np
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
import time
import seaborn as sns
from torch.distributions import Categorical

pag.FAILSAFE = False


import argparse
import torch

import torch.multiprocessing as _mp
from torch.distributions import Categorical
import torch.nn.functional as F


import numpy as np
import shutil
from torch.utils.tensorboard import SummaryWriter


from random import sample

from pynput import keyboard
from pynput import mouse
import threading
import tkinter as tk
from tkinter import messagebox

In [40]:
#env.py
class ENV():
    def __init__(self):
        # screenshot의 위치 지정, 클래스 생성할때 가져오기

        # winflg와 lose flag 존재해야 함. 0으로 하는 건 grayscale
        self.winFlag = cv2.imread('/home/jnu/Desktop/Reinforce/Royale/screenshot/cr_test/weWin.png', 0)
        self.loseFlag = cv2.imread('/home/jnu/Desktop/Reinforce/Royale/screenshot/cr_test/enemyWin.png', 0)
        # nocard flag
        self.nocardFlag = cv2.imread('/home/jnu/Desktop/Reinforce/Royale/screenshot/test1/nocard.png')
        # noElixir flag
        self.noelixirFlag = cv2.imread('/home/jnu/Desktop/Reinforce/Royale/screenshot/test1/noElixir.png')
        self.startGameFlag = cv2.imread('/home/jnu/Desktop/Reinforce/Royale/screenshot/cr_test/battleStart.png')
        self.finishFlag = cv2.imread('/home/jnu/Desktop/Reinforce/Royale/screenshot/cr_test/battleFinish.png')
        self.enemy1Flag = cv2.imread('/home/jnu/Desktop/Reinforce/Royale/screenshot/cr_test/enemy1.png')
        self.enemy2Flag = cv2.imread('/home/jnu/Desktop/Reinforce/Royale/screenshot/cr_test/enemy2.png')
        self.enemy3Flag = cv2.imread('/home/jnu/Desktop/Reinforce/Royale/screenshot/cr_test/enemy3.png')
        self.stop_flag = cv2.imread('/home/jnu/Desktop/Reinforce/Royale/screenshot/cr_test/stop_game_full.png')
        self.main_flag = cv2.imread('/home/jnu/Desktop/Reinforce/Royale/screenshot/cr_test/main_check.png')
        
    def return_state(self, img):
        # 스크린 샷을 인자로 받아와서 모델에 넣을 수 있도록 tensor로 변환
        #tf = transforms
        img_t = tf(img) # time.sleep(0.3)
        img_t = img_t.unsqueeze(0)
        # img_t = img_t.permute(1, 0, 2, 3)

        return img_t

    def check_finish(self,img):
        finishMessage = cv2.cvtColor(self.finishFlag, cv2.COLOR_BGR2GRAY)
        img = np.array(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ratio = cv2.matchTemplate(finishMessage, img, cv2.TM_CCOEFF_NORMED)
        if (np.max(ratio) > 0.90):
            return 1

        else:
            return 0

    def check_win(self, img):
        # 게임이 이겼는지 확인, screenshot을 가져와서 우리가 원하는 크기로 잘라서 확인
        # img = np.array(img)
        checkFlag1 = np.array(img.crop((225, 335, 280, 365)))
        checkFlag1 = cv2.cvtColor(checkFlag1, cv2.COLOR_BGR2GRAY)
        win_check = cv2.matchTemplate(checkFlag1, self.winFlag, cv2.TM_CCOEFF_NORMED)
        if win_check > 0.8:
            return 1

        else:
            return 0
           
    def check_lose(self, img):
        # 게임이 졌는지 확인, screenshot을 가져와서 우리가 원하는 크기로 잘라서 확인
        # img = np.array(img)
        checkFlag2 = np.array(img.crop((225, 85, 280, 115)))
        checkFlag2 = cv2.cvtColor(checkFlag2, cv2.COLOR_BGR2GRAY)
        lose_check = cv2.matchTemplate(checkFlag2, self.loseFlag, cv2.TM_CCOEFF_NORMED)
        if lose_check > 0.8:
            return 1

        else:
            return 0

    def check_card(self, img):
        # 카드를 선택하지 않았는지 확인, screenshot을 가져와서 init에 지정된 nocard 이미지와 비교하여 reward 부여
        nocard = cv2.cvtColor(self.nocardFlag, cv2.COLOR_BGR2GRAY)
        img = np.array(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ratio = cv2.matchTemplate(nocard, img, cv2.TM_CCOEFF_NORMED)
        #print(np.max(ratio))
        if (np.max(ratio) > 0.90):
            # print(np.max(ratio))
            return 1

        else:
            return 0

    def check_elixir(self, img):
        noElixir = cv2.cvtColor(self.noelixirFlag, cv2.COLOR_BGR2GRAY)
        img = np.array(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ratio = cv2.matchTemplate(noElixir, img, cv2.TM_CCOEFF_NORMED)

        if (np.max(ratio) > 0.90):
            # print(np.max(ratio))
            return 1

        else:
            return 0

    def checkET1(self, img):
        score1 = 0
        checkFlag1 = np.array(img.crop((105, 135, 155, 136)))

        for i in range(50):
            if (checkFlag1[0][i][0] >= 96):
                score1 += 1

        score1 = score1 * 2
        return score1

    def checkET2(self, img):
        score2 = 0
        checkFlag2 = np.array(img.crop((371, 135, 421, 136)))

        for i in range(50):
            if (checkFlag2[0][i][0] >= 96):
                score2 += 1

        score2 = score2 * 2
        return score2

    def checkGameStart(self, img):
        startMessage = cv2.cvtColor(self.startGameFlag, cv2.COLOR_BGR2GRAY)
        img = np.array(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ratio = cv2.matchTemplate(startMessage, img, cv2.TM_CCOEFF_NORMED)
        if (np.max(ratio) > 0.90):
            # print(np.max(ratio))
            return 1

        else:
            return 0
        
    def checkMainPage(self, img):
        main_check = cv2.cvtColor(self.main_flag, cv2.COLOR_BGR2GRAY)
        img = np.array(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ratio = cv2.matchTemplate(main_check, img, cv2.TM_CCOEFF_NORMED)
        #win_lose_flagprint(np.max(ratio))
        if (np.max(ratio) > 0.90):
            # print(np.max(ratio))
            return 1

        else:
            return 0
        
    def stop_game(self, img):
        stopMessage = cv2.cvtColor(self.stop_flag, cv2.COLOR_BGR2GRAY)
        img = np.array(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ratio = cv2.matchTemplate(stopMessage, img, cv2.TM_CCOEFF_NORMED)
        #print(np.max(ratio))
        if (np.max(ratio) > 0.90):
            # print(np.max(ratio))
            return 1

        else:
            return 0

    def enemy1(self, img):
        crownFlag1 = cv2.cvtColor(self.enemy1Flag, cv2.COLOR_BGR2GRAY)
        img1 = np.array(img)
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        ratio = cv2.matchTemplate(crownFlag1, img1, cv2.TM_CCOEFF_NORMED)
        #print(np.max(ratio))
        if (np.max(ratio) > 0.90):
            return 1
        else:
            return 0
        
    def enemy2(self, img):
        crownFlag2 = cv2.cvtColor(self.enemy2Flag, cv2.COLOR_BGR2GRAY)
        img1 = np.array(img)
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        ratio = cv2.matchTemplate(crownFlag2, img1, cv2.TM_CCOEFF_NORMED)
        #print(np.max(ratio))
        if (np.max(ratio) > 0.90):
            return 1
        else:
            return 0
        
    def enemy3(self, img):
        crownFlag3 = cv2.cvtColor(self.enemy3Flag, cv2.COLOR_BGR2GRAY)
        img1 = np.array(img)
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        ratio = cv2.matchTemplate(crownFlag3, img1, cv2.TM_CCOEFF_NORMED)
        #print(np.max(ratio))
        if (np.max(ratio) > 0.90):
            return 1
        else:
            return 0

    # def checkOT1
    # def checkOT1
    # def checkOT1

    # 우리 타워와 상대 타워의 hp를 확인하여 reward 부여

    def retryGame(self):
        time.sleep(8)
        pag.click((2860, 875))
        #print("step1 click")
        time.sleep(5)
        
        while(1):
            img = pag.screenshot(region=(2605, 100, 510, 900))
            if(self.checkMainPage(img)):
                break
            time.sleep(5)
                
        pag.click((3070, 185))
        time.sleep(5)
        pag.click((2920, 385))
        time.sleep(3)
        pag.click((2950, 615))


In [41]:
env = ENV()

In [42]:
def do_acion():
    return random.randrange(1, 46)

In [43]:
rewardbox = [-1, -1, -1, -1, -1, -1, -1, -1, 20, -3, 12]
def get_reward():
    return random.choice(rewardbox)

In [44]:
img = pag.screenshot(region=(2605, 100, 510, 900))
s = env.return_state(img).cuda()

TypeError: 'module' object is not callable

In [45]:
s

tensor([[[[0.2078, 0.2078, 0.2078,  ..., 0.2078, 0.2078, 0.2078],
          [0.2078, 0.2078, 0.2078,  ..., 0.2078, 0.2078, 0.2078],
          [0.2078, 0.2078, 0.2078,  ..., 0.2078, 0.2078, 0.2078],
          ...,
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],

         [[0.2118, 0.2118, 0.2118,  ..., 0.2118, 0.2118, 0.2118],
          [0.2118, 0.2118, 0.2118,  ..., 0.2118, 0.2118, 0.2118],
          [0.2118, 0.2118, 0.2118,  ..., 0.2118, 0.2118, 0.2118],
          ...,
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],

         [[0.2275, 0.2275, 0.2275,  ..., 0.2275, 0.2275, 0.2275],
          [0.2275, 0.2275, 0.2275,  ..., 0.2275, 0.2275, 0.2275],
          [0.2275, 0.2275, 0.2275,  ..., 0

In [46]:
random.randrange(1, 46)

21

In [47]:
import pickle
import pandas as pd

In [50]:
data = []
data_pre = []

In [51]:
count = 0
while(count <100):
    
    s = pag.screenshot(region=(2605, 100, 510, 900))
    
    a = do_acion()
    s_prime = pag.screenshot(region=(2605, 100, 510, 900))
    
    reward = get_reward()
    done = 0
    data_pre.append((s, a, s_prime, reward, done))
    count +=1
#     dapa.append(data_pre)
#     data_pre = []

In [38]:
data_pre

[(tensor([[[[0.2078, 0.2078, 0.2078,  ..., 0.2078, 0.2078, 0.2078],
            [0.2078, 0.2078, 0.2078,  ..., 0.2078, 0.2078, 0.2078],
            [0.2078, 0.2078, 0.2078,  ..., 0.2078, 0.2078, 0.2078],
            ...,
            [0.9529, 0.9529, 0.9529,  ..., 0.9529, 0.9529, 0.9529],
            [0.9529, 0.9529, 0.9529,  ..., 0.9529, 0.9529, 0.9529],
            [0.9529, 0.9529, 0.9529,  ..., 0.9529, 0.9529, 0.9529]],
  
           [[0.2118, 0.2118, 0.2118,  ..., 0.2118, 0.2118, 0.2118],
            [0.2118, 0.2118, 0.2118,  ..., 0.2118, 0.2118, 0.2118],
            [0.2118, 0.2118, 0.2118,  ..., 0.2118, 0.2118, 0.2118],
            ...,
            [0.9529, 0.9529, 0.9529,  ..., 0.9529, 0.9529, 0.9529],
            [0.9529, 0.9529, 0.9529,  ..., 0.9529, 0.9529, 0.9529],
            [0.9529, 0.9529, 0.9529,  ..., 0.9529, 0.9529, 0.9529]],
  
           [[0.2275, 0.2275, 0.2275,  ..., 0.2275, 0.2275, 0.2275],
            [0.2275, 0.2275, 0.2275,  ..., 0.2275, 0.2275, 0.2275],
      

In [52]:
len(data_pre)

100

In [53]:
np.save('test_file', np.array(data_pre))

/tmp/ipykernel_47882/355026662.py:1: FutureWarning: The input object of type 'Image' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Image', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  np.save('test_file', np.array(data_pre))
/tmp/ipykernel_47882/355026662.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.save('test_file', np.array(data_pre))


In [55]:
s = pag.screenshot(region=(2605, 100, 510, 900))
# s = np.array(s)
# s_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

error: OpenCV(4.7.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'


In [57]:
img = cv2.imread('./screenshot/cr_test/play_screen.png')
img2 = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [59]:
cv2.imwrite('./screenshot/cr_test/play_screen_gray.png', img2)


True

In [60]:
img3 = cv2.resize(img2, (0, 0), fx = 0.5, fy = 0.5)

In [61]:
cv2.imwrite('./screenshot/cr_test/play_screen_resize_0.5.png', img3)

True

In [ ]:
img4 = cv2.